In [23]:
import reedsolo

def rs_encode(data, nsym):
    """
    Encodes data using Reed-Solomon code.
    
    :param data: The data to encode (bytes)
    :param nsym: The number of error correction symbols to add
    :return: Encoded data with error correction symbols appended
    """
    rs = reedsolo.RSCodec(nsym)
    encoded_data = rs.encode(data)
    return encoded_data

def rs_decode(encoded_data, nsym):
    """
    Decodes Reed-Solomon encoded data.
    
    :param encoded_data: The encoded data with errors (bytes)
    :param nsym: The number of error correction symbols used during encoding
    :return: Decoded data with errors corrected
    """
    rs = reedsolo.RSCodec(nsym)
    decoded_data = rs.decode(encoded_data)
    return decoded_data

def simulate_noise(encoded_data, error_count):
    """
    Simulates a noisy channel by introducing errors into the encoded data.
    
    :param encoded_data: The encoded data (bytes)
    :param error_count: The number of errors to introduce
    :return: Encoded data with errors
    """
    import random
    noisy_data = bytearray(encoded_data)
    for _ in range(error_count):
        pos = random.randint(0, len(noisy_data) - 1)
        noisy_data[pos] ^= random.randint(1, 255)  # Introduce a random error
    return noisy_data

def max_correctable_symbols(nsym):
    """
    Calculates the maximum number of symbols that can be corrected.
    
    :param nsym: The number of error correction symbols used during encoding
    :return: The maximum number of correctable symbols
    """
    return nsym // 2

if __name__ == "__main__":
    # Example usage
    message = b"Hello, this is a test message"
    nsym = 10  # Number of error correction symbols

    print("Original message:", message)

    # Encode the message
    encoded_message = rs_encode(message, nsym)
    print("Encoded message:", encoded_message)

    # Calculate maximum correctable symbols
    max_symbols = max_correctable_symbols(nsym)
    print("Maximum number of correctable symbols:", max_symbols)

    # Simulate a noisy channel
    noisy_message = simulate_noise(encoded_message, error_count=max_symbols)
    print("Noisy message:", noisy_message)

    # Decode the noisy message
    try:
        decoded_message = rs_decode(noisy_message, nsym)
        print("Decoded message:", decoded_message)
    except reedsolo.ReedSolomonError as e:
        print("Decoding failed:", e)


Original message: b'Hello, this is a test message'
Encoded message: bytearray(b'Hello, this is a test message\xe8\xf6ud\x1a\xe6\xfd\xaa)\xc3')
Maximum number of correctable symbols: 5
Noisy message: bytearray(b'Hell\xa6, t\x9disvis a test messag\xb3\xe8\xf6Td\x1a\xe6\xfd\xaa)\xc3')
Decoded message: (bytearray(b'Hello, this is a test message'), bytearray(b'Hello, this is a test message\xe8\xf6ud\x1a\xe6\xfd\xaa)\xc3'), bytearray(b'\x1f\x1c\x0b\x08\x04'))


In [1]:
import numpy as np

# Define the GF(2^8) field with a primitive polynomial (x^8 + x^4 + x^3 + x^2 + 1)
# which corresponds to the binary representation 100011101
def gf_add(x, y):
    return x ^ y  # Addition in GF(2^8) is simply XOR

def gf_sub(x, y):
    return x ^ y  # Subtraction in GF(2^8) is the same as addition

def gf_mult(x, y, prim=0x11d, field_charac_full=256):
    r = 0
    while y:
        if y & 1:
            r ^= x
        y >>= 1
        x <<= 1
        if x & field_charac_full:
            x ^= prim
    return r

def gf_div(x, y, prim=0x11d, field_charac_full=256):
    if y == 0:
        raise ZeroDivisionError()
    if x == 0:
        return 0
    # Inverse of y in GF(2^8)
    y_inv = gf_inv(y, prim, field_charac_full)
    return gf_mult(x, y_inv, prim, field_charac_full)

def gf_inv(x, prim=0x11d, field_charac_full=256):
    # Extended Euclidean Algorithm to find the inverse of x in GF(2^8)
    lm, hm = 1, 0
    low, high = x % field_charac_full, prim
    while low > 1:
        r = high // low
        nm, new = hm - lm * r, high - low * r
        lm, low, hm, high = nm, new, lm, low
    return lm % field_charac_full

def gf_poly_add(p, q):
    r = [0] * max(len(p), len(q))
    for i in range(len(p)):
        r[i + len(r) - len(p)] = p[i]
    for i in range(len(q)):
        r[i + len(r) - len(q)] ^= q[i]
    return r

def gf_poly_mult(p, q, prim=0x11d):
    r = [0] * (len(p) + len(q) - 1)
    for j in range(len(q)):
        for i in range(len(p)):
            r[i + j] ^= gf_mult(p[i], q[j], prim)
    return r

def gf_poly_div(dividend, divisor):
    # Polynomial long division in GF(2^8)
    msg_out = list(dividend)  # Copy of dividend
    for i in range(len(dividend) - len(divisor) + 1):
        coef = msg_out[i]
        if coef != 0:
            for j in range(1, len(divisor)):
                if divisor[j] != 0:
                    msg_out[i + j] ^= gf_mult(divisor[j], coef)
    separator = -(len(divisor) - 1)
    return msg_out[:separator], msg_out[separator:]  # quotient, remainder

def rs_generator_poly(nsym, prim=0x11d):
    g = [1]
    for i in range(nsym):
        g = gf_poly_mult(g, [1, gf_pow(2, i)], prim)
    return g

def gf_pow(x, power, prim=0x11d, field_charac_full=256):
    r = 1
    for _ in range(power):
        r = gf_mult(r, x, prim, field_charac_full)
    return r

def rs_encode_msg(msg_in, nsym, prim=0x11d):
    if len(msg_in) + nsym > 255:
        raise ValueError("Message too long")
    gen = rs_generator_poly(nsym, prim)
    msg_out = [0] * (len(msg_in) + nsym)
    msg_out[:len(msg_in)] = msg_in
    for i in range(len(msg_in)):
        coef = msg_out[i]
        if coef != 0:
            for j in range(1, len(gen)):
                msg_out[i + j] ^= gf_mult(gen[j], coef, prim)
    msg_out[:len(msg_in)] = msg_in
    return msg_out

# Example usage
if __name__ == "__main__":
    message = [ord(c) for c in "Hello, this is a test message"]
    nsym = 10  # Number of error correction symbols

    print("Original message:", message)

    # Encode the message
    encoded_message = rs_encode_msg(message, nsym)
    print("Encoded message:", encoded_message)


Original message: [72, 101, 108, 108, 111, 44, 32, 116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 116, 101, 115, 116, 32, 109, 101, 115, 115, 97, 103, 101]
Encoded message: [72, 101, 108, 108, 111, 44, 32, 116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 116, 101, 115, 116, 32, 109, 101, 115, 115, 97, 103, 101, 232, 246, 117, 100, 26, 230, 253, 170, 41, 195]


In [2]:
def gf_add(x, y):
    return x ^ y  # Penjumlahan di GF(2^8) adalah XOR

def gf_mult(x, y, prim=0x11d, field_charac_full=256):
    r = 0
    while y:
        if y & 1:
            r ^= x
        y >>= 1
        x <<= 1
        if x & field_charac_full:
            x ^= prim
    return r

def gf_pow(x, power, prim=0x11d, field_charac_full=256):
    r = 1
    for _ in range(power):
        r = gf_mult(r, x, prim, field_charac_full)
    return r

def gf_poly_mult(p, q, prim=0x11d):
    r = [0] * (len(p) + len(q) - 1)
    for j in range(len(q)):
        for i in range(len(p)):
            r[i + j] ^= gf_mult(p[i], q[j], prim)
    return r

def rs_generator_poly(nsym, prim=0x11d):
    g = [1]
    for i in range(nsym):
        g = gf_poly_mult(g, [1, gf_pow(2, i)], prim)
    return g

def rs_encode_msg(msg_in, nsym, prim=0x11d):
    if len(msg_in) + nsym > 255:
        raise ValueError("Pesan terlalu panjang")
    gen = rs_generator_poly(nsym, prim)
    msg_out = [0] * (len(msg_in) + nsym)
    msg_out[:len(msg_in)] = msg_in
    for i in range(len(msg_in)):
        coef = msg_out[i]
        if coef != 0:
            for j in range(1, len(gen)):
                msg_out[i + j] ^= gf_mult(gen[j], coef, prim)
    msg_out[:len(msg_in)] = msg_in
    return msg_out, gen  # Mengembalikan pesan yang dikodekan dan polinomial generator

# Contoh penggunaan
if __name__ == "__main__":
    # Konversi pesan ke nilai ASCII
    message = [ord(c) for c in "Hello, this is a test message"]
    nsym = 10  # Jumlah simbol koreksi kesalahan

    print("Pesan asli:", message)

    # Kodekan pesan dan dapatkan polinomial generator
    encoded_message, generator_poly = rs_encode_msg(message, nsym)
    
    print("Pesan yang dikodekan:", encoded_message)
    print("Polinomial Generator:", generator_poly)


Pesan asli: [72, 101, 108, 108, 111, 44, 32, 116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 116, 101, 115, 116, 32, 109, 101, 115, 115, 97, 103, 101]
Pesan yang dikodekan: [72, 101, 108, 108, 111, 44, 32, 116, 104, 105, 115, 32, 105, 115, 32, 97, 32, 116, 101, 115, 116, 32, 109, 101, 115, 115, 97, 103, 101, 232, 246, 117, 100, 26, 230, 253, 170, 41, 195]
Polinomial Generator: [1, 216, 194, 159, 111, 199, 94, 95, 113, 157, 193]
